<a href="https://colab.research.google.com/github/isshii/de4test/blob/test_ozawa/DeepXplore/DeepXplore_99_out_func_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 共通で使うパスなどの定義
# 共通の変数設定
# 共通フォルダパス

import datetime
import pytz
dt_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
dt_str = str(dt_now.strftime('%Y%m%d_%H%M'))

data_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data"
#data_imagenet = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data/ImageNet"
data_mnist = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data/MNIST"
#data_imagenet_seeds = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data/ImageNet/seeds"
model_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/model"
#output_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/output"
output_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/output/" + dt_str
tmp_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/tmp"

In [2]:
# Goggle Drive つなぐ
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import sys
import argparse
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format

%tensorflow_version 1.x
from keras.datasets import mnist
from keras.layers import Input
import imageio

from __future__ import print_function

from keras.datasets import mnist
from keras.layers import Convolution2D, MaxPooling2D, Input, Dense, Activation, Flatten
from keras.models import Model
from keras.utils import to_categorical
import os

# TensorFlowでGPUを使っているかのチェック：
# "device_type: "GPU" " があればOK
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

Using TensorFlow backend.


In [0]:
#@title デフォルトのタイトル テキスト
# DeepXplore のパラメータ設定部
# read the parameter
# argument parsing
parser = argparse.ArgumentParser(description='Main function for difference-inducing input generation in MNIST dataset')
parser.add_argument('transformation', help="realistic transformation type", choices=['light', 'occl', 'blackout'])
parser.add_argument('weight_diff', help="weight hyperparm to control differential behavior", type=float)
parser.add_argument('weight_nc', help="weight hyperparm to control neuron coverage", type=float)
parser.add_argument('step', help="step size of gradient descent", type=float)
parser.add_argument('seeds', help="number of seeds of input", type=int)
parser.add_argument('grad_iterations', help="number of iterations of gradient descent", type=int)
parser.add_argument('threshold', help="threshold for determining neuron activated", type=float)
parser.add_argument('-t', '--target_model', help="target model that we want it predicts differently",
                    choices=[0, 1, 2], default=0, type=int)
parser.add_argument('-sp', '--start_point', help="occlusion upper left corner coordinate", default=(0, 0), type=tuple)
parser.add_argument('-occl_size', '--occlusion_size', help="occlusion size", default=(10, 10), type=tuple)

#@title ノイズのタイプ light / occl / blackout
#@body {light,occl,blackout} weight_diff weight_nc step seeds
#                   grad_iterations threshold
noise_type = "light" #@param ["light", "occl", "blackout"]
weight_diff = "0.1" #@param {type:"string"}
weight_nc = "0.1" #@param {type:"string"}
step = "5" #@param {type:"string"}　#50だとほぼ白飛び
seeds = "10" #@param {type:"string"}
grad_iterations = "40" #@param {type:"string"}
threshold = "0.1" #@param {type:"string"}

args = parser.parse_args([noise_type, weight_diff, weight_nc, step, seeds, grad_iterations, threshold])

In [0]:
import random
from collections import defaultdict

import numpy as np
from keras import backend as K
from keras.models import Model


# util function to convert a tensor into a valid image
def deprocess_image(x):
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x.reshape(x.shape[1], x.shape[2])  # original shape (1,img_rows, img_cols,1)


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)


def constraint_occl(gradients, start_point, rect_shape):
    new_grads = np.zeros_like(gradients)
    new_grads[:, start_point[0]:start_point[0] + rect_shape[0],
    start_point[1]:start_point[1] + rect_shape[1]] = gradients[:, start_point[0]:start_point[0] + rect_shape[0],
                                                     start_point[1]:start_point[1] + rect_shape[1]]
    return new_grads


def constraint_light(gradients):
    new_grads = np.ones_like(gradients)
    grad_mean = np.mean(gradients)
    return grad_mean * new_grads


def constraint_black(gradients, rect_shape=(6, 6)):
    start_point = (
        random.randint(0, gradients.shape[1] - rect_shape[0]), random.randint(0, gradients.shape[2] - rect_shape[1]))
    new_grads = np.zeros_like(gradients)
    patch = gradients[:, start_point[0]:start_point[0] + rect_shape[0], start_point[1]:start_point[1] + rect_shape[1]]
    if np.mean(patch) < 0:
        new_grads[:, start_point[0]:start_point[0] + rect_shape[0],
        start_point[1]:start_point[1] + rect_shape[1]] = -np.ones_like(patch)
    return new_grads


def init_coverage_tables(model1, model2, model3):
    model_layer_dict1 = defaultdict(bool)
    model_layer_dict2 = defaultdict(bool)
    model_layer_dict3 = defaultdict(bool)
    init_dict(model1, model_layer_dict1)
    init_dict(model2, model_layer_dict2)
    init_dict(model3, model_layer_dict3)
    return model_layer_dict1, model_layer_dict2, model_layer_dict3


def init_dict(model, model_layer_dict):
    for layer in model.layers:
        if 'flatten' in layer.name or 'input' in layer.name:
            continue
        for index in range(layer.output_shape[-1]):
            model_layer_dict[(layer.name, index)] = False


def neuron_to_cover(model_layer_dict):
    not_covered = [(layer_name, index) for (layer_name, index), v in model_layer_dict.items() if not v]
    if not_covered:
        layer_name, index = random.choice(not_covered)
    else:
        layer_name, index = random.choice(model_layer_dict.keys())
    return layer_name, index


def neuron_covered(model_layer_dict):
    covered_neurons = len([v for v in model_layer_dict.values() if v])
    total_neurons = len(model_layer_dict)
    return covered_neurons, total_neurons, covered_neurons / float(total_neurons)


def update_coverage(input_data, model, model_layer_dict, threshold=0):
    layer_names = [layer.name for layer in model.layers if
                   'flatten' not in layer.name and 'input' not in layer.name]

    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=[model.get_layer(layer_name).output for layer_name in layer_names])
    intermediate_layer_outputs = intermediate_layer_model.predict(input_data)

    for i, intermediate_layer_output in enumerate(intermediate_layer_outputs):
        scaled = scale(intermediate_layer_output[0])
        for num_neuron in range(scaled.shape[-1]):
            if np.mean(scaled[..., num_neuron]) > threshold and not model_layer_dict[(layer_names[i], num_neuron)]:
                model_layer_dict[(layer_names[i], num_neuron)] = True


def full_coverage(model_layer_dict):
    if False in model_layer_dict.values():
        return False
    return True


def scale(intermediate_layer_output, rmax=1, rmin=0):
    X_std = (intermediate_layer_output - intermediate_layer_output.min()) / (
        intermediate_layer_output.max() - intermediate_layer_output.min())
    X_scaled = X_std * (rmax - rmin) + rmin
    return X_scaled


def fired(model, layer_name, index, input_data, threshold=0):
    intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    intermediate_layer_output = intermediate_layer_model.predict(input_data)[0]
    scaled = scale(intermediate_layer_output)
    if np.mean(scaled[..., index]) > threshold:
        return True
    return False


def diverged(predictions1, predictions2, predictions3, target):
    #     if predictions2 == predictions3 == target and predictions1 != target:
    if not predictions1 == predictions2 == predictions3:
        return True
    return False


#add
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

In [6]:
'''
LeNet-1
'''
def Model1(input_tensor=None, train=False):
    nb_classes = 10
    # convolution kernel size
    kernel_size = (5, 5)

    if train:
        batch_size = 256
        nb_epoch = 1

        # input image dimensions
        img_rows, img_cols = 28, 28

        # the data, shuffled and split between train and test sets
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        print(x_train.shape)
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255

        # convert class vectors to binary class matrices
        y_train = to_categorical(y_train, nb_classes)
        y_test = to_categorical(y_test, nb_classes)

        input_tensor = Input(shape=input_shape)
    elif input_tensor is None:
        print(bcolors.FAIL + 'you have to proved input_tensor when testing')
        exit()

    # block1
    x = Convolution2D(4, kernel_size, activation='relu', padding='same', name='block1_conv1')(input_tensor)
    x = MaxPooling2D(pool_size=(2, 2), name='block1_pool1')(x)

    # block2
    x = Convolution2D(12, kernel_size, activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='block2_pool1')(x)

    x = Flatten(name='flatten')(x)
    x = Dense(nb_classes, name='before_softmax')(x)
    x = Activation('softmax', name='predictions')(x)

    model = Model(input_tensor, x)

    if train:
        # compiling
        model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

        # trainig
        model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=nb_epoch, verbose=1)
        # save model
#        model.save_weights('./Model1.h5')
        model.save_weights('/content/gdrive/My Drive/ColabNotebooks/test4ai/model/MNIST/Model1.h5')
        score = model.evaluate(x_test, y_test, verbose=0)
        print('\n')
        print('Overall Test score:', score[0])
        print('Overall Test accuracy:', score[1])
    else:
#        model.load_weights('./Model1.h5')
        model.load_weights('/content/gdrive/My Drive/ColabNotebooks/test4ai/model/MNIST/Model1.h5')
        print(bcolors.OKBLUE + 'Model1 loaded' + bcolors.ENDC)

    return model


if __name__ == '__main__':
    Model1(train=True)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/1





60000/60000 [==============================] - 15s 249us/step - loss: 0.4600 - acc: 0.8627 - val_loss: 0.1193 - val_acc: 0.9654


Overall Test score: 0.11932606396973133
Overall Test accuracy: 0.9654


In [7]:
# input MNIST image dimensions 
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)
input_tensor = Input(shape=input_shape)

#define the model instance
model1 = Model1(input_tensor=input_tensor)
model2 = Model1(input_tensor=input_tensor)
model3 = Model1(input_tensor=input_tensor)

#define the dictionary of neuron coverage
model_layer_dict1, model_layer_dict2, model_layer_dict3 = init_coverage_tables(model1, model2, model3)

Model1 loaded
Model1 loaded
Model1 loaded


In [20]:
num_fig = 10
def create_data(start_fig, num_fi, length):
#  (datax_train, y_train), (x_test, y_test) = mnist.load_data()
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  test_per_fig_x = np.array([])
  test_per_fig_y = np.array([])
  tests_x = np.array([])
  tests_y = np.array([])

  x_train = x_train.astype('float32')
  x_test  = x_test.astype('float32')
  x_train = x_train.astype('float')
  x_test = x_test.astype('float')
  x_train /= 255
  x_test /= 255
  #length = int(args.seeds/num_fig)
  #length = int(args.seeds)
  for i in range(start_fig, start_fig+num_fig):
  #i=0
    cond = [(x==i) for x in y_test]
    test_per_fig_x = x_test[cond]
    test_per_fig_y = y_test[cond]
    print("figure:", i, ", shape:", test_per_fig_x.shape, ", deviation:",test_per_fig_x.shape[0]-1000)
    tests_x = np.append(tests_x, test_per_fig_x[:length])
    tests_y = np.append(tests_y, test_per_fig_y[:length])  

  #conds = [conds, cond]
  print("check!", tests_x.shape[0]/img_rows/img_cols/num_fig, "=", length, "equal?")
  tests_x = tests_x.reshape(-1,img_rows, img_cols,1)
  tests_x.shape
  tests_x = tests_x.astype('float32')
  return tests_x, tests_y, length

tests_x, tests_y, length = create_data(0, 10, int(args.seeds))
  
model = model1
input_data =  np.expand_dims(tests_x[(length*0 + 0)], axis=0)

figure: 0 , shape: (980, 28, 28) , deviation: -20
figure: 1 , shape: (1135, 28, 28) , deviation: 135
figure: 2 , shape: (1032, 28, 28) , deviation: 32
figure: 3 , shape: (1010, 28, 28) , deviation: 10
figure: 4 , shape: (982, 28, 28) , deviation: -18
figure: 5 , shape: (892, 28, 28) , deviation: -108
figure: 6 , shape: (958, 28, 28) , deviation: -42
figure: 7 , shape: (1028, 28, 28) , deviation: 28
figure: 8 , shape: (974, 28, 28) , deviation: -26
figure: 9 , shape: (1009, 28, 28) , deviation: 9
check! 10.0 = 10 equal?


In [0]:
layer_names = [layer.name for layer in model.layers if 'flatten' not in layer.name and 'input' not in layer.name]

intermediate_layer_model = Model(inputs=model.input, outputs=[model.get_layer(layer_name).output for layer_name in layer_names])
intermediate_layer_outputs = intermediate_layer_model.predict(input_data)
    
out_list = []
out_list_scale = []
for i, intermediate_layer_output in enumerate(intermediate_layer_outputs):
  scaled = scale(intermediate_layer_output[0])
  for num_neuron in range(scaled.shape[-1]):
    out_list.append(np.mean(intermediate_layer_output[..., num_neuron]))
    out_list_scale.append(np.mean(scaled[..., num_neuron]))

In [42]:
intermediate_layer_outputs[2].shape

(1, 14, 14, 12)

In [44]:
intermediate_layer_outputs[2][0].shape

(14, 14, 12)

In [46]:
scaled = scale(intermediate_layer_outputs[2][0])
scaled.shape

(14, 14, 12)

In [48]:
scaled.shape[-1]

12

In [59]:
temp = 0
for i in range(scaled.shape[-1]):
  temp += np.mean(scaled[..., i]) 
  print(i, temp, np.mean(scaled[..., i]))

0 0.06580380350351334 0.0658038
1 0.11581691727042198 0.050013114
2 0.18674978986382484 0.07093287
3 0.23490989953279495 0.04816011
4 0.29656488448381424 0.061654985
5 0.39496809244155884 0.09840321
6 0.4643409252166748 0.06937283
7 0.5324339270591736 0.068093
8 0.6125428676605225 0.08010894
9 0.6459387019276619 0.033395834
10 0.6989351436495781 0.05299644
11 0.7640047296881676 0.065069586


In [61]:
for i in range(6):
  print(intermediate_layer_outputs[i].shape)

(1, 28, 28, 4)
(1, 14, 14, 4)
(1, 14, 14, 12)
(1, 7, 7, 12)
(1, 10)
(1, 10)


In [60]:
print(np.max(scaled))

1.0


In [62]:
intermediate_layer_outputs[5].shape

(1, 10)

In [63]:
scaled = scale(intermediate_layer_outputs[5][0])
scaled.shape

(10,)

In [64]:
scaled.shape[-1]

10

In [65]:
temp = 0
for i in range(scaled.shape[-1]):
  temp += np.mean(scaled[..., i]) 
  print(i, temp, np.mean(scaled[..., i]))

0 1.0 1.0
1 1.0 0.0
2 1.0000730086685508 7.300867e-05
3 1.0000734821446713 4.7347612e-07
4 1.000073630421383 1.4827671e-07
5 1.0000809440197997 7.3135984e-06
6 1.0002617787785937 0.00018083476
7 1.0002623340930938 5.553145e-07
8 1.0002646686802734 2.3345872e-06
9 1.000273149807498 8.481127e-06
